# ECE367: PS02 Part 2 -- PageRank

## Framing

$N$ is the number of URL's. $J$ is the adjacency matrix.

### Power Iteration Method

* Method for calculating eigenvalues/vectors for diagonalizable matrix. 
* 

## Steps

- [x] Load the pagerank data from `pagerank_urls.txt`, `pagerank_adj.mat`.
- [x] Based on adjacency matrix $J$ calculate $$A_{i, j} = \frac{J_{i, j}}{\sum_{k=1}^{N}J_{k,j}}$$
    - [x] Verify that the rows add to 1.
- [x] Implement the **power iteration method** (OptM 7.1.1) for 10 iterations.
    - [x] Calculate $e(k+1) = ||Ax(k+1) - x(k+1)||_2$
    - [x] Plot $\log(e(k+1))$ vs. k
- [ ] Implement **shift-invert power iteration** and **Rayleigh quotient iteration** algorithms (OptM 7.1.2, 7.1.3). 
    - [ ] For shift-invert: $\sigma = 0.99$.
    - [ ] For Rayleigh quotient: $\sigma_1 = \sigma_2 = 0.99$ for first two iterations.
        - [ ] $\sigma_k = \frac{x^*(k) Ax(k)}{x^*(k)x(k)}$ for k > 2.
    - [ ] Plot $\log(e(k+1))$ vs. k for each plot.
- [ ] List the (page index, PageRank score) tuples for **top 5** and **bottom 5** pages according to PageRank scores.

In [2]:
# IMPORT BOX #
# IMPORT BOX #
using Plots
using Plotly
using GR
using SymPy
using MAT
using LinearAlgebra

plotly()

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-5490180560862812783\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-1857378428935753588\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-5490180560862812783\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

┌ Info: For saving to png with the Plotly backend PlotlyBase has to be installed.
└ @ Plots /Users/abhargava/.julia/packages/Plots/a7Kbp/src/backends.jl:373


Plots.PlotlyBackend()

In [3]:
file_J = matopen("PS02_dataSet/pagerank_adj.mat")
J = read(file_J)["J"];
typeof(J)

Array{Float64,2}

In [4]:
# Normalization function for J
function normalize_J(J::Array{Float64,2})
    A = zeros(size(J))
    
    for i = 1:size(J,1)
        if(sum(J[:,i]) == 0)
            println("Sum of row ",i," equals 0...")
        else
            A[:,i] = J[:,i]/sum(J[:,i])
        end
        
    end
    A
end

normalize_J (generic function with 1 method)

In [5]:
A = normalize_J(J);
x = sum(A, dims=1);

In [6]:
good_sum = true

for i = 1:size(x,1)
    if abs(x[i] - 1) > 0.00001
        good_sum = false
        println("incorrect sum at ",i," -- x[i] = ",x[i])
    end
end

if good_sum
    println("Sum of A's columns are all 1. Checks out")
else
    println("Sum of A's columns are not all 1.")
end

x = transpose(x); # Turning x into a column vector

Sum of A's columns are all 1. Checks out


In [7]:
function get_err(A::Array{Float64,2}, x)
    norm(A*x - x)
end

get_err (generic function with 1 method)

In [8]:
function power_iteration(A::Array{Float64,2}, xin, num_iters)
    x = copy(xin)
    if norm(x) != 1
        x = x./norm(x)
    end
    
    errors = zeros(num_iters)
    
    λ = 0   
    for k = 1:num_iters
        y = A*x
        x = y/norm(y)
        λ = transpose(x)*A*x
        errors[k] = get_err(A, x)
    end
    
    return (x, errors)
end

power_iteration (generic function with 1 method)

In [9]:
function shift_invert_power_iteration(A::Array{Float64,2}, xin, num_iters, σ=0.99)
    x = copy(xin)
    if norm(x) != 1
        x = x./norm(x)
    end
    
    errors = zeros(num_iters)
    
    λ = 0   
    n = size(A,1)
    for k = 1:num_iters
        y = inv(A-(σ.*I(n)))*x
        x = y/norm(y)
        λ = transpose(x)*A*x
        errors[k] = get_err(A, x)
    end
    
    return (x, errors)
end

shift_invert_power_iteration (generic function with 2 methods)

In [10]:
function rayleigh_quotient_iteration(A::Array{Float64,2}, xin, num_iters, σ=0.99)
    x = copy(xin)
    
    if norm(x) != 1
        x = x./norm(x)
    end

    errors = zeros(num_iters)
    λ = 0
    n = size(A,1)
    for k = 1:num_iters
        if k > 2
            σ = (transpose(x)*A*x)/(transpose(x)*x)
        end
        y = inv(A-(σ.*I(n)))*x
        x = y/norm(y)
        λ = transpose(x)*A*x
        errors[k] = get_err(A, x)
    end

    return (x, errors)
end

rayleigh_quotient_iteration (generic function with 2 methods)

In [11]:
x_pi, errors_pi = power_iteration(A, x, 10);
x_sipi, errors_sipi = shift_invert_power_iteration(A, x, 10);
x_rqi, errors_rqi = rayleigh_quotient_iteration(A, x, 10);

In [12]:
Plots.plot(log10.(errors_pi), label="e(k+1)")
Plots.title!("Power Iteration: Error vs. Iteration")
Plots.xlabel!("k")
Plots.ylabel!("log e(k+1)")
Plots.savefig("figures/2_9_PI_err_it.png")

In [16]:
Plots.plot(log10.(errors_sipi), label="e(k+1)")
Plots.title!("Shift-Invert Power Iteration: Error vs. Iteration")
Plots.xlabel!("k")
Plots.ylabel!("e(k+1)")
Plots.savefig("figures/2_9_SIPI_err_it.png")

In [14]:
Plots.plot(log10.(errors_rqi), label="e(k+1)")
Plots.title!("Rayleigh Quotient Iteration: Error vs. Iteration")
Plots.xlabel!("k")
Plots.ylabel!("log e(k+1)")
Plots.savefig("figures/2_9_RQI_err_it.png")

In [230]:
# Using the text file, we report the most 'important' pages
namefile = open("PS02_dataSet/pagerank_urls.txt");
noms = readlines(namefile);

In [250]:
function get_top_n(x, n, absolute=false)
    x_cpy = copy(x)
    
    if absolute
        x_cpy = abs.(x_cpy)
    end
    
    indexes = zeros(Int16, n)
    
    for i = 1:n
        mxval, mxindx = findmax(x_cpy)
        idx = mxindx[1]
        indexes[i] = convert(Int16, idx)
        
        x_cpy[idx] = 0
    end
    
    indexes
end
function get_bottom_n(x, n, absolute=false)
    x_cpy = copy(x)
    
    if absolute
        x_cpy = abs.(x_cpy)
    end
    
    indexes = zeros(Int16, n)
    
    for i = 1:n
        mxval, mxindx = findmin(x_cpy)
        idx = mxindx[1]
        indexes[i] = convert(Int16, idx)
        
        x_cpy[idx] = 1
    end
    
    indexes
end

get_bottom_n (generic function with 2 methods)

In [251]:
idxs = get_top_n(x_pi, 5)
println("For Power Iteration")
for i = 1:5
    println("Top ",i," Page: ",noms[idxs[i]]," at index ",idxs[i]," with score ",x_pi[idxs[i]])
end
println("--")
idxs = get_bottom_n(x_pi, 5)
for i = 1:5
    println("Bottom ",i," Page: ",noms[idxs[i]]," at index ",idxs[i]," with score ",x_pi[idxs[i]])
end

For Power Iteration
Top 1 Page: http://www.hollins.edu/ at index 2 with score 0.4393669395950616
Top 2 Page: http://www.hollins.edu/admissions/visit/visit.htm at index 35 with score 0.3295553218972332
Top 3 Page: http://www.hollins.edu/about/about_tour.htm at index 36 with score 0.3072679100390794
Top 4 Page: http://www.hollins.edu/htdig/index.html at index 58 with score 0.29959997614868467
Top 5 Page: http://www.hollins.edu/admissions/info-request/info-request.cfm at index 49 with score 0.27509160711857744
--
Bottom 1 Page: http://www1.hollins.edu/ at index 1 with score 0.0
Bottom 2 Page: http://www1.hollins.edu/Docs/Forms/GetForms.htm at index 3 with score 0.0
Bottom 3 Page: http://www1.hollins.edu/Docs/misc/travel.htm at index 4 with score 0.0
Bottom 4 Page: http://www1.hollins.edu/Docs/GVCalendar/gvmain.htm at index 5 with score 0.0
Bottom 5 Page: http://www1.hollins.edu/Docs/comptech/comptech.htm at index 10 with score 0.0


In [252]:
idxs = get_top_n(x_sipi, 5)
println("For Shift-Invert Power Iteration (Not Taking Absolute Values)")
for i = 1:5
    println("Top ",i," Page: ",noms[idxs[i]]," at index ",idxs[i]," with score ",x_sipi[idxs[i]])
end
println("--")
idxs = get_bottom_n(x_sipi, 5)
for i = 1:5
    println("Bottom ",i," Page: ",noms[idxs[i]]," at index ",idxs[i]," with score ",x_sipi[idxs[i]])
end

idxs = get_top_n(x_sipi, 5, true)
println("\n\n\nFor Shift-Invert Power Iteration (Absolute Values)")
for i = 1:5
    println("Top ",i," Page: ",noms[idxs[i]]," at index ",idxs[i]," with score ",x_sipi[idxs[i]])
end
println("--")
idxs = get_bottom_n(x_sipi, 5)
for i = 1:5
    println("Bottom ",i," Page: ",noms[idxs[i]]," at index ",idxs[i]," with score ",x_sipi[idxs[i]])
end

For Shift-Invert Power Iteration (Not Taking Absolute Values)
Top 1 Page: http://www.hollins.edu/ at index 2 with score 0.371076800177363
Top 2 Page: http://www.hollins.edu/admissions/visit/visit.htm at index 35 with score 0.3184300571822739
Top 3 Page: http://www.hollins.edu/about/about_tour.htm at index 36 with score 0.2974898527981775
Top 4 Page: http://www.hollins.edu/htdig/index.html at index 58 with score 0.2903683424743939
Top 5 Page: http://www.hollins.edu/admissions/info-request/info-request.cfm at index 49 with score 0.26374804470798907
--
Bottom 1 Page: http://www1.hollins.edu/homepages/hammerpw/qrhomepage.htm at index 424 with score -0.3196296934984905
Bottom 2 Page: http://www1.hollins.edu/homepages/hammerpw/qrcourses2.htm at index 987 with score -0.16021771066678325
Bottom 3 Page: http://www1.hollins.edu/homepages/hammerpw/qrcourses.htm at index 986 with score -0.16019371004811028
Bottom 4 Page: http://www1.hollins.edu/homepages/hammerpw/qractivities.htm at index 985 with

In [253]:
idxs = get_top_n(x_rqi, 5)
println("For Rayleigh Quotient Iteration")
for i = 1:5
    println("Top ",i," Page: ",noms[idxs[i]]," at index ",idxs[i]," with score ", x_rqi[idxs[i]])
end
println("--")
idxs = get_bottom_n(x_rqi, 5)
for i = 1:5
    println("Bottom ",i," Page: ",noms[idxs[i]]," at index ",idxs[i]," with score ", x_rqi[idxs[i]])
end

For Rayleigh Quotient Iteration
Top 1 Page: http://www1.hollins.edu/homepages/hammerpw/qrhomepage.htm at index 424 with score 0.663327738197711
Top 2 Page: http://www1.hollins.edu/homepages/hammerpw/qrcourses.htm at index 986 with score 0.33166386909885553
Top 3 Page: http://www1.hollins.edu/homepages/hammerpw/qrcourses2.htm at index 987 with score 0.3316638690988555
Top 4 Page: http://www1.hollins.edu/homepages/hammerpw/qractivities.htm at index 985 with score 0.24874790182414167
Top 5 Page: http://www1.hollins.edu/homepages/hammerpw/qrgrantsummary.htm at index 984 with score 0.16583193454942774
--
Bottom 1 Page: http://www1.hollins.edu/ at index 1 with score 3.713492277236943e-74
Bottom 2 Page: http://www.hollins.edu/academics/library/libtoc.htm at index 48 with score 3.713492277236943e-74
Bottom 3 Page: http://www1.hollins.edu/Docs/GVCalendar/gvmain.htm at index 5 with score 5.404751202091903e-74
Bottom 4 Page: http://www1.hollins.edu/Docs/Forms/GetForms.htm at index 3 with score 5.

In [254]:
get_err(A, x_pi)

0.11297845011565136

In [245]:
get_err(A, x_sipi)

0.000783822615251815

In [246]:
get_err(A, x_rqi)

1.6786171296150374e-16

In [18]:
Plots.plot(x_pi, label="Power Iteration")
Plots.plot!(x_sipi, label="Shift-Inverte Power Method")
Plots.plot!(x_rqi, label="Rayleigh")
Plots.savefig("guhguh")

In [238]:
det([1 1; 1 1])

0.0